# Week 8: 중단 가능한 문서 처리 파이프라인

## 과제 개요
- **주제**: 안전하고 신뢰할 수 있는 파이프라인 구축
- **목표**: 중단/재개 가능한 4단계 작업 처리 시스템
- **핵심**: 체크포인트 + 파일 저장 + 대기열

## 7주차 vs 8주차

| 항목 | 7주차 | 8주차 |
|------|-------|-------|
| 실행 방식 | 처음~끝 연속 | 단계별 중단 가능 |
| 상태 저장 | 메모리만 | 메모리 + 파일 |
| 복원 | 불가능 | 가능 (resume) |
| 동시 요청 | 미지원 | 대기열 처리 |

## 비유

- **7주차**: "논스톱 영화 상영"
  - 한 번 시작하면 끝까지  
  
- **8주차**: "챕터별 DVD 재생"
  - 챕터 1 → 저장
  - 챕터 2 → 저장
  - 중단 → 나중에 챕터 3부터 재개
  
## 4단계 파이프라인사용자 요청  
↓  
Step 1: 요구사항 분석 → 📁 step_01.json  
↓  
Step 2: 정보 수집 → 📁 step_02.json  
↓  
Step 3: 콘텐츠 생성 → 📁 step_03.json  
↓  
Step 4: 최종 검증 → 📁 step_04.json  
↓  
최종 결과 → 📁 final_result.json  
  
## 구현 내용

1. **Section 0**: 환경 설정 + 폴더 구조
2. **Section 1**: 4단계 파이프라인 노드
3. **Section 2**: 파일 I/O 시스템
4. **Section 3**: 체크포인트 (STOP/resume)
5. **Section 4**: 대기열 시스템
6. **Section 5**: 시나리오 테스트

In [1]:
import os
import json
import uuid
from datetime import datetime
from pathlib import Path
from typing import TypedDict, Annotated, Dict, Any, Optional, List
import warnings
warnings.filterwarnings('ignore')

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 파일 I/O 및 대기열
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from queue import Queue
from threading import Lock

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# LangChain Core (7주차 재사용)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# LangGraph (7주차 재사용 + 체크포인터)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages

print("✅ 라이브러리 Import 완료")

print("""
[8주차 새로운 Import]
  - json: JSON 파일 저장/로드
  - pathlib.Path: 파일 경로 관리
  - queue.Queue: 대기열
  - threading.Lock: 동시 실행 제어
  
[7주차에서 재사용]
  - ChatOpenAI, PromptTemplate
  - StateGraph, MemorySaver
  - add_messages
""")

✅ 라이브러리 Import 완료

[8주차 새로운 Import]
  - json: JSON 파일 저장/로드
  - pathlib.Path: 파일 경로 관리
  - queue.Queue: 대기열
  - threading.Lock: 동시 실행 제어

[7주차에서 재사용]
  - ChatOpenAI, PromptTemplate
  - StateGraph, MemorySaver
  - add_messages



In [2]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")
print("✅ OpenAI API 키 로드 완료")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# LLM 설정
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.5  # 중간 창의성

llm = ChatOpenAI(
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    api_key=openai_api_key
)

print(f"✅ LLM 설정 완료: {LLM_MODEL}")

print("""
[LLM 설정]
  - temperature: 0.5 (균형잡힌 출력)
  - 7주차(0.7)보다 낮춤 → 일관성 중시
""")

✅ OpenAI API 키 로드 완료
✅ LLM 설정 완료: gpt-4o-mini

[LLM 설정]
  - temperature: 0.5 (균형잡힌 출력)
  - 7주차(0.7)보다 낮춤 → 일관성 중시



In [3]:
BASE_DIR = Path(".")
RUNS_DIR = BASE_DIR / "runs"

# runs 폴더 생성
RUNS_DIR.mkdir(exist_ok=True)

print(f"✅ 기본 폴더 구조 생성 완료")
print(f"  BASE_DIR: {BASE_DIR.absolute()}")
print(f"  RUNS_DIR: {RUNS_DIR.absolute()}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# run_id별 폴더 생성 함수
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def create_run_folder(run_id: str) -> Path:
    """run_id별 폴더 생성
    
    Args:
        run_id: 작업 ID
        
    Returns:
        run 폴더 경로
    """
    run_folder = RUNS_DIR / run_id
    run_folder.mkdir(exist_ok=True)
    
    print(f"  ✅ Run 폴더 생성: {run_folder}")
    
    return run_folder

print("\n✅ 폴더 구조 함수 정의 완료")

print("""
[폴더 구조]
  project/
  ├── runs/
  │   ├── run_abc123/
  │   │   ├── step_01.json
  │   │   ├── step_02.json
  │   │   ├── step_03.json
  │   │   ├── step_04.json
  │   │   ├── checkpoint.json
  │   │   └── final_result.json
  │   └── run_def456/
  │       └── ...
  └── pipeline.ipynb
""")

# 테스트
test_run_id = "test_" + uuid.uuid4().hex[:8]
test_folder = create_run_folder(test_run_id)
print(f"\n[테스트] 폴더 생성: {test_folder}")

✅ 기본 폴더 구조 생성 완료
  BASE_DIR: /Users/joyoungha/Desktop/project/rag-practice
  RUNS_DIR: /Users/joyoungha/Desktop/project/rag-practice/runs

✅ 폴더 구조 함수 정의 완료

[폴더 구조]
  project/
  ├── runs/
  │   ├── run_abc123/
  │   │   ├── step_01.json
  │   │   ├── step_02.json
  │   │   ├── step_03.json
  │   │   ├── step_04.json
  │   │   ├── checkpoint.json
  │   │   └── final_result.json
  │   └── run_def456/
  │       └── ...
  └── pipeline.ipynb

  ✅ Run 폴더 생성: runs/test_48cbfb64

[테스트] 폴더 생성: runs/test_48cbfb64


In [4]:
# 4단계 정의
PIPELINE_STEPS = {
    1: "요구사항 분석",
    2: "정보 수집",
    3: "콘텐츠 생성",
    4: "최종 검증"
}

# 파일명 규칙
STEP_FILE_TEMPLATE = "step_{:02d}.json"
CHECKPOINT_FILE = "checkpoint.json"
FINAL_RESULT_FILE = "final_result.json"

# 작업 상태
class JobStatus:
    """작업 상태"""
    PENDING = "pending"      # 대기 중
    RUNNING = "running"      # 실행 중
    STOPPED = "stopped"      # 중단됨
    COMPLETED = "completed"  # 완료
    FAILED = "failed"        # 실패

print("✅ 상수 정의 완료")

print(f"""
[4단계 파이프라인]
  Step 1: {PIPELINE_STEPS[1]}
  Step 2: {PIPELINE_STEPS[2]}
  Step 3: {PIPELINE_STEPS[3]}
  Step 4: {PIPELINE_STEPS[4]}

[파일명 규칙]
  - 단계별: {STEP_FILE_TEMPLATE.format(1)}, {STEP_FILE_TEMPLATE.format(2)}, ...
  - 체크포인트: {CHECKPOINT_FILE}
  - 최종 결과: {FINAL_RESULT_FILE}

[작업 상태]
  - PENDING: 대기 중
  - RUNNING: 실행 중
  - STOPPED: 중단됨 (resume 가능)
  - COMPLETED: 완료
  - FAILED: 실패
""")

✅ 상수 정의 완료

[4단계 파이프라인]
  Step 1: 요구사항 분석
  Step 2: 정보 수집
  Step 3: 콘텐츠 생성
  Step 4: 최종 검증

[파일명 규칙]
  - 단계별: step_01.json, step_02.json, ...
  - 체크포인트: checkpoint.json
  - 최종 결과: final_result.json

[작업 상태]
  - PENDING: 대기 중
  - RUNNING: 실행 중
  - STOPPED: 중단됨 (resume 가능)
  - COMPLETED: 완료
  - FAILED: 실패



In [5]:
class PipelineState(TypedDict):
    """파이프라인 State
    
    Attributes:
        run_id: 작업 고유 ID
        user_goal: 사용자 요청
        current_step: 현재 단계 (1-4)
        completed_steps: 완료된 단계 리스트
        step_outputs: 각 단계 출력 결과
        status: 작업 상태
        stop_requested: 중단 요청 플래그
        timestamp: 타임스탬프
    """
    run_id: str
    user_goal: str
    current_step: int
    completed_steps: List[int]
    step_outputs: Dict[str, Any]  # {"step_1": {...}, "step_2": {...}}
    status: str
    stop_requested: bool
    timestamp: str

print("✅ PipelineState 정의 완료")

print("""
[PipelineState 필드]

1. run_id
   - 작업 고유 ID (UUID)
   - 폴더명으로 사용

2. user_goal
   - 사용자의 원본 요청
   - 예: "AI 트렌드 보고서 작성"

3. current_step
   - 현재 진행 중인 단계 (1-4)
   - 0: 시작 전, 4: 완료

4. completed_steps
   - 완료된 단계 리스트
   - 예: [1, 2] → Step 1, 2 완료

5. step_outputs
   - 각 단계의 출력 결과 저장
   - {"step_1": {...}, "step_2": {...}}

6. status
   - 작업 상태 (pending/running/stopped/completed)

7. stop_requested
   - 중단 요청 플래그
   - True면 다음 단계로 넘어가지 않음

8. timestamp
   - 마지막 업데이트 시간
""")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 초기 State 생성 함수
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def create_initial_state(user_goal: str) -> PipelineState:
    """초기 PipelineState 생성
    
    Args:
        user_goal: 사용자 요청
        
    Returns:
        초기화된 PipelineState
    """
    run_id = "run_" + uuid.uuid4().hex[:8]
    
    return {
        "run_id": run_id,
        "user_goal": user_goal,
        "current_step": 0,
        "completed_steps": [],
        "step_outputs": {},
        "status": JobStatus.PENDING,
        "stop_requested": False,
        "timestamp": datetime.now().isoformat()
    }

print("\n✅ 초기 State 생성 함수 정의 완료")

# 테스트
test_state = create_initial_state("AI 트렌드 분석 보고서")
print(f"\n[테스트] 초기 State:")
print(f"  run_id: {test_state['run_id']}")
print(f"  user_goal: {test_state['user_goal']}")
print(f"  current_step: {test_state['current_step']}")
print(f"  status: {test_state['status']}")

✅ PipelineState 정의 완료

[PipelineState 필드]

1. run_id
   - 작업 고유 ID (UUID)
   - 폴더명으로 사용

2. user_goal
   - 사용자의 원본 요청
   - 예: "AI 트렌드 보고서 작성"

3. current_step
   - 현재 진행 중인 단계 (1-4)
   - 0: 시작 전, 4: 완료

4. completed_steps
   - 완료된 단계 리스트
   - 예: [1, 2] → Step 1, 2 완료

5. step_outputs
   - 각 단계의 출력 결과 저장
   - {"step_1": {...}, "step_2": {...}}

6. status
   - 작업 상태 (pending/running/stopped/completed)

7. stop_requested
   - 중단 요청 플래그
   - True면 다음 단계로 넘어가지 않음

8. timestamp
   - 마지막 업데이트 시간


✅ 초기 State 생성 함수 정의 완료

[테스트] 초기 State:
  run_id: run_4c796365
  user_goal: AI 트렌드 분석 보고서
  current_step: 0
  status: pending


# Section 1: 4단계 파이프라인 노드 구현
---

## 목표
- 4개 파이프라인 노드 구현
- 각 노드는 명확한 역할
- LLM을 활용한 단계별 처리

## 4단계 파이프라인 상세

### Step 1: 요구사항 분석
**역할**: 사용자 요청을 분석하여 작업 계획 수립

입력: user_goal
출력: {
  "analysis": "요구사항 분석 결과",
  "plan": "단계별 실행 계획",
  "key_points": ["핵심 포인트 1", "핵심 포인트 2"]
}

예시:
- 입력: "AI 트렌드 보고서 작성"
- 출력: "2025년 AI 트렌드를 분석하고, LLM/멀티모달/에이전트 등 핵심 주제를 다루는 보고서 작성 계획"

---

### Step 2: 정보 수집
**역할**: 필요한 정보/데이터 수집

입력: Step 1 분석 결과
출력: {
  "collected_data": "수집된 정보",
  "sources": ["출처 1", "출처 2"],
  "summary": "요약"
}

예시:
- 입력: "AI 트렌드 분석 계획"
- 출력: "GPT-5, Claude 4, 멀티모달 AI 확산 등 2025년 주요 트렌드 정보"

---

### Step 3: 콘텐츠 생성
**역할**: 수집된 정보를 바탕으로 콘텐츠 작성

입력: Step 1 계획 + Step 2 데이터
출력: {
  "content": "작성된 콘텐츠",
  "structure": "구조 설명",
  "word_count": 단어 수
}

예시:
- 입력: "AI 트렌드 정보"
- 출력: "# 2025 AI 트렌드 보고서\n\n## 1. LLM의 진화\n..."

---

### Step 4: 최종 검증
**역할**: 품질 검증 및 개선 제안

입력: Step 3 콘텐츠
출력: {
  "validation": "검증 결과",
  "quality_score": 점수,
  "improvements": ["개선점 1", "개선점 2"],
  "final_status": "approved" | "needs_revision"
}

예시:
- 입력: "작성된 보고서"
- 출력: "품질 점수 8.5/10, 통계 자료 추가 권장"

In [6]:
step1_template = """당신은 요구사항 분석 전문가입니다.

사용자 요청: {user_goal}

위 요청을 분석하여 다음을 포함한 작업 계획을 수립하세요:

1. 요구사항 분석
   - 사용자가 원하는 것이 무엇인가?
   - 어떤 형식의 결과물을 원하는가?

2. 실행 계획
   - Step 2에서 수집할 정보는?
   - Step 3에서 생성할 콘텐츠 구조는?
   - Step 4에서 검증할 항목은?

3. 핵심 포인트
   - 꼭 포함해야 할 내용 (3-5개)

출력 형식 (JSON):
{{
  "analysis": "요구사항 분석 결과",
  "plan": "단계별 실행 계획",
  "key_points": ["핵심 포인트 1", "핵심 포인트 2", "핵심 포인트 3"]
}}

분석 결과:"""

step1_prompt = PromptTemplate(
    template=step1_template,
    input_variables=["user_goal"]
)

def step_1_node(state: PipelineState) -> PipelineState:
    """Step 1: 요구사항 분석
    
    Args:
        state: 현재 파이프라인 State
        
    Returns:
        업데이트된 State
    """
    print("\n" + "="*80)
    print(f"[Step 1] {PIPELINE_STEPS[1]}")
    print("="*80)
    print(f"Run ID: {state['run_id']}")
    print(f"요청: {state['user_goal']}")
    
    # LLM으로 분석
    print(f"\n  [1/2] 요구사항 분석 중...")
    
    analysis_text = (step1_prompt | llm | StrOutputParser()).invoke({
        "user_goal": state["user_goal"]
    })
    
    # JSON 파싱
    import re
    json_match = re.search(r'\{.*\}', analysis_text, re.DOTALL)
    if json_match:
        try:
            analysis_result = json.loads(json_match.group())
        except:
            analysis_result = {
                "analysis": analysis_text,
                "plan": "단계별 처리",
                "key_points": ["품질", "정확성", "완성도"]
            }
    else:
        analysis_result = {
            "analysis": analysis_text,
            "plan": "단계별 처리",
            "key_points": ["품질", "정확성", "완성도"]
        }
    
    print(f"  ✅ 분석 완료")
    print(f"     핵심 포인트: {len(analysis_result.get('key_points', []))}개")
    
    # State 업데이트
    print(f"\n  [2/2] State 업데이트 중...")
    
    step_outputs = state.get("step_outputs", {})
    step_outputs["step_1"] = analysis_result
    
    completed_steps = state.get("completed_steps", [])
    if 1 not in completed_steps:
        completed_steps.append(1)
    
    print(f"  ✅ Step 1 완료")
    
    return {
        "current_step": 1,
        "completed_steps": completed_steps,
        "step_outputs": step_outputs,
        "status": JobStatus.RUNNING,
        "timestamp": datetime.now().isoformat()
    }

print("✅ Step 1 노드 구현 완료")

✅ Step 1 노드 구현 완료


In [7]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 웹 검색 도구 (7주차 재사용)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily API 키 확인
tavily_api_key = os.getenv("TAVILY_API_KEY")
if tavily_api_key:
    web_search_tool = TavilySearchResults(
        max_results=5,
        api_key=tavily_api_key
    )
    print("✅ Tavily 웹 검색 도구 로드 완료")
else:
    # Fallback: 더미 도구
    def dummy_search(query):
        return [{
            "content": f"'{query}'에 대한 검색 결과 (더미 데이터)",
            "url": "https://example.com"
        }]
    web_search_tool = type('obj', (object,), {'invoke': dummy_search})()
    print("⚠️ Tavily API 키 없음 - 더미 도구 사용")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Step 2 프롬프트 (웹 검색 결과 활용)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

step2_template = """당신은 정보 수집 전문가입니다.

사용자 요청: {user_goal}

Step 1 분석 결과:
{step1_result}

웹 검색 결과:
{search_results}

위 웹 검색 결과를 바탕으로 정보를 **상세히** 정리하세요.

중요: Step 3에서 이 정보를 활용하여 콘텐츠를 작성할 것이므로, 
가능한 한 구체적이고 풍부한 정보를 제공해야 합니다.

요구사항:

1. 수집된 정보 (collected_data)
   - 각 출처별로 핵심 내용을 **상세히** 정리
   - 구체적인 사례, 통계, 인용구 포함
   - 최소 800-1000자 분량
   - 다음 구조로 작성:
     
     [주제 1: Step 1의 첫 번째 핵심 포인트]
     - 출처 1에서: (구체적 내용, 통계, 사례)
     - 출처 2에서: (구체적 내용, 통계, 사례)
     
     [주제 2: Step 1의 두 번째 핵심 포인트]
     - 출처 3에서: (구체적 내용, 통계, 사례)
     - 출처 4에서: (구체적 내용, 통계, 사례)
     
     [주제 3: Step 1의 세 번째 핵심 포인트]
     - 출처 5에서: (구체적 내용, 통계, 사례)

2. 출처 (sources)
   - 실제 검색된 URL 목록
   - 각 URL이 다루는 주제 간단히 명시

3. 요약 (summary)
   - 전체 수집 정보의 핵심만 2-3문장으로

출력 형식 (JSON):
{{
  "collected_data": "주제별로 구조화된 상세 정보 (800-1000자)",
  "sources": [
    {{"url": "URL1", "topic": "다루는 주제"}},
    {{"url": "URL2", "topic": "다루는 주제"}}
  ],
  "summary": "핵심 요약 2-3문장"
}}

정리 결과:"""

step2_prompt = PromptTemplate(
    template=step2_template,
    input_variables=["user_goal", "step1_result", "search_results"]
)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Step 2 노드 함수 (웹 검색 포함)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def step_2_node(state: PipelineState) -> PipelineState:
    """Step 2: 정보 수집 (실제 웹 검색)
    
    Args:
        state: 현재 파이프라인 State
        
    Returns:
        업데이트된 State
    """
    print("\n" + "="*80)
    print(f"[Step 2] {PIPELINE_STEPS[2]}")
    print("="*80)
    
    # Step 1 결과 가져오기
    step1_result = state["step_outputs"]["step_1"]
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 1. 웹 검색 수행
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    print(f"\n  [1/3] 웹 검색 중...")
    
    # 검색 쿼리 생성 (user_goal 기반)
    search_query = state["user_goal"]
    
    print(f"     검색어: '{search_query}'")
    
    try:
        # 실제 웹 검색 실행
        search_results_raw = web_search_tool.invoke(search_query)
        
        # 검색 결과 포맷팅
        search_results_text = ""
        for i, result in enumerate(search_results_raw[:5], 1):
            content = result.get("content", "")
            url = result.get("url", "")
            search_results_text += f"\n[{i}] {content}\n출처: {url}\n"
        
        print(f"     ✅ 검색 완료: {len(search_results_raw)}개 결과")
        
    except Exception as e:
        print(f"     ⚠️ 웹 검색 실패: {e}")
        search_results_text = "웹 검색을 수행할 수 없습니다. 일반 지식을 활용합니다."
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 2. LLM으로 정보 정리
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    print(f"\n  [2/3] 정보 정리 중...")
    
    collection_text = (step2_prompt | llm | StrOutputParser()).invoke({
        "user_goal": state["user_goal"],
        "step1_result": json.dumps(step1_result, ensure_ascii=False, indent=2),
        "search_results": search_results_text
    })
    
    # JSON 파싱
    import re
    json_match = re.search(r'\{.*\}', collection_text, re.DOTALL)
    if json_match:
        try:
            collection_result = json.loads(json_match.group())
        except:
            collection_result = {
                "collected_data": collection_text,
                "sources": ["웹 검색 결과 기반"],
                "summary": "정보 수집 완료"
            }
    else:
        collection_result = {
            "collected_data": collection_text,
            "sources": ["웹 검색 결과 기반"],
            "summary": "정보 수집 완료"
        }
    
    # 실제 검색 결과 URL 추가 (LLM이 환각하는 경우 대비)
    if search_results_raw:
        actual_urls = [r.get("url", "") for r in search_results_raw[:5] if r.get("url")]
        collection_result["sources"] = actual_urls[:5]  # 최대 5개
    
    print(f"  ✅ 정리 완료")
    print(f"     출처: {len(collection_result.get('sources', []))}개")
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 3. State 업데이트
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    print(f"\n  [3/3] State 업데이트 중...")
    
    step_outputs = state["step_outputs"]
    step_outputs["step_2"] = collection_result
    
    completed_steps = state["completed_steps"]
    if 2 not in completed_steps:
        completed_steps.append(2)
    
    print(f"  ✅ Step 2 완료")
    
    return {
        "current_step": 2,
        "completed_steps": completed_steps,
        "step_outputs": step_outputs,
        "timestamp": datetime.now().isoformat()
    }

print("✅ Step 2 노드 구현 완료 (실제 웹 검색 포함)")

✅ Tavily 웹 검색 도구 로드 완료
✅ Step 2 노드 구현 완료 (실제 웹 검색 포함)


In [8]:
step3_template = """당신은 콘텐츠 작성 전문가입니다.

사용자 요청: {user_goal}

Step 1 분석:
{step1_result}

Step 2 수집 정보:
{step2_result}

위 정보를 바탕으로 콘텐츠를 작성하세요.

요구사항:
1. Step 1의 계획에 따라 구조화
2. Step 2의 수집 정보 활용
3. 읽기 쉽고 명확한 표현
4. 적절한 길이 (300-500 단어)

출력 형식 (JSON):
{{
  "content": "작성된 콘텐츠 전문",
  "structure": "구조 설명 (예: 서론-본론-결론)",
  "word_count": 단어수
}}

작성 결과:"""

step3_prompt = PromptTemplate(
    template=step3_template,
    input_variables=["user_goal", "step1_result", "step2_result"]
)

def step_3_node(state: PipelineState) -> PipelineState:
    """Step 3: 콘텐츠 생성
    
    Args:
        state: 현재 파이프라인 State
        
    Returns:
        업데이트된 State
    """
    print("\n" + "="*80)
    print(f"[Step 3] {PIPELINE_STEPS[3]}")
    print("="*80)
    
    # Step 1, 2 결과 가져오기
    step1_result = state["step_outputs"]["step_1"]
    step2_result = state["step_outputs"]["step_2"]
    
    print(f"  참고: Step 1 분석 + Step 2 정보")
    
    # LLM으로 콘텐츠 생성
    print(f"\n  [1/2] 콘텐츠 생성 중...")
    
    content_text = (step3_prompt | llm | StrOutputParser()).invoke({
        "user_goal": state["user_goal"],
        "step1_result": json.dumps(step1_result, ensure_ascii=False, indent=2),
        "step2_result": json.dumps(step2_result, ensure_ascii=False, indent=2)
    })
    
    # JSON 파싱
    import re
    json_match = re.search(r'\{.*\}', content_text, re.DOTALL)
    if json_match:
        try:
            content_result = json.loads(json_match.group())
        except:
            content_result = {
                "content": content_text,
                "structure": "단일 구조",
                "word_count": len(content_text.split())
            }
    else:
        content_result = {
            "content": content_text,
            "structure": "단일 구조",
            "word_count": len(content_text.split())
        }
    
    print(f"  ✅ 생성 완료")
    print(f"     단어 수: {content_result.get('word_count', 0)}개")
    
    # State 업데이트
    print(f"\n  [2/2] State 업데이트 중...")
    
    step_outputs = state["step_outputs"]
    step_outputs["step_3"] = content_result
    
    completed_steps = state["completed_steps"]
    if 3 not in completed_steps:
        completed_steps.append(3)
    
    print(f"  ✅ Step 3 완료")
    
    return {
        "current_step": 3,
        "completed_steps": completed_steps,
        "step_outputs": step_outputs,
        "timestamp": datetime.now().isoformat()
    }

print("✅ Step 3 노드 구현 완료")

✅ Step 3 노드 구현 완료


In [ ]:
step4_template = """당신은 엄격한 품질 검증 전문가입니다.

사용자 요청: {user_goal}

Step 3 작성 콘텐츠:
{step3_result}

위 콘텐츠를 **매우 엄격하게** 검증하세요.

중요: 점수는 보수적으로 평가하세요. 
- 우수한 경우에만 8점 이상
- 평범하면 5-7점
- 문제가 있으면 4점 이하

검증 항목 (각 1-10점):

1. 요구사항 충족도 (30%)
   - 사용자가 요청한 내용을 정확히 다루는가?
   - 요청한 형식(에세이/보고서 등)에 맞는가?
   - 요청한 분량에 맞는가?
   
   점수 기준:
   - 9-10점: 모든 요구사항 완벽히 충족
   - 7-8점: 대부분 충족, 사소한 누락
   - 5-6점: 일부 충족, 중요 내용 누락
   - 3-4점: 요구사항 불충분
   - 1-2점: 요구사항 거의 불충족

2. 내용 품질 (40%)
   - 구체적인 사례/통계가 포함되었는가?
   - 논리적 흐름이 명확한가?
   - 깊이 있는 분석인가, 피상적인가?
   - Step 2의 정보를 충분히 활용했는가?
   
   점수 기준:
   - 9-10점: 풍부한 사례, 깊이 있는 분석, 논리 완벽
   - 7-8점: 적절한 사례, 논리적이나 깊이 부족
   - 5-6점: 사례 부족, 일반론적 서술
   - 3-4점: 구체성 없음, 논리 불명확
   - 1-2점: 내용 부실

3. 가독성 (30%)
   - 문장이 명확하고 자연스러운가?
   - 단락 구성이 적절한가?
   - 전문 용어 사용이 적절한가?
   
   점수 기준:
   - 9-10점: 매우 읽기 쉬움, 구성 완벽
   - 7-8점: 읽기 쉬움, 구성 적절
   - 5-6점: 이해 가능하나 개선 필요
   - 3-4점: 읽기 어려움
   - 1-2점: 이해 불가

최종 판단 기준:
- 가중 평균 7.5점 이상: approved (승인)
- 가중 평균 5.0-7.4점: needs_major_revision (대폭 수정 필요)
- 가중 평균 5.0점 미만: needs_complete_rewrite (전면 재작성)

개선 제안:
- 구체적으로 무엇이 부족한지 명시
- 어떤 내용을 추가해야 하는지 제안
- 최소 3가지 이상의 실행 가능한 개선점

출력 형식 (JSON):
{{
  "validation": "검증 결과 상세 설명 (각 항목별 평가)",
  "scores": {{
    "requirements": 점수,
    "quality": 점수,
    "readability": 점수
  }},
  "weighted_score": 가중평균,
  "improvements": ["구체적 개선점 1", "구체적 개선점 2", "구체적 개선점 3"],
  "final_status": "approved | needs_major_revision | needs_complete_rewrite"
}}

검증 결과:"""

step4_prompt = PromptTemplate(
    template=step4_template,
    input_variables=["user_goal", "step3_result"]
)

def step_4_node(state: PipelineState) -> PipelineState:
    """Step 4: 최종 검증
    
    Args:
        state: 현재 파이프라인 State
        
    Returns:
        업데이트된 State
    """
    print("\n" + "="*80)
    print(f"[Step 4] {PIPELINE_STEPS[4]}")
    print("="*80)
    
    # Step 3 결과 가져오기
    step3_result = state["step_outputs"]["step_3"]
    
    print(f"  참고: Step 3 콘텐츠")
    
    # LLM으로 검증
    print(f"\n  [1/2] 품질 검증 중...")
    
    validation_text = (step4_prompt | llm | StrOutputParser()).invoke({
        "user_goal": state["user_goal"],
        "step3_result": json.dumps(step3_result, ensure_ascii=False, indent=2)
    })
    
    # JSON 파싱
    import re
    json_match = re.search(r'\{.*\}', validation_text, re.DOTALL)
    if json_match:
        try:
            validation_result = json.loads(json_match.group())
        except:
            validation_result = {
                "validation": validation_text,
                "quality_score": 8.0,
                "improvements": [],
                "final_status": "approved"
            }
    else:
        validation_result = {
            "validation": validation_text,
            "quality_score": 8.0,
            "improvements": [],
            "final_status": "approved"
        }
    
    print(f"  ✅ 검증 완료")
    print(f"     품질 점수: {validation_result.get('quality_score', 0)}/10")
    print(f"     상태: {validation_result.get('final_status', 'unknown')}")
    
    # State 업데이트
    print(f"\n  [2/2] State 업데이트 중...")
    
    step_outputs = state["step_outputs"]
    step_outputs["step_4"] = validation_result
    
    completed_steps = state["completed_steps"]
    if 4 not in completed_steps:
        completed_steps.append(4)
    
    print(f"  ✅ Step 4 완료")
    print(f"  ✅ 전체 파이프라인 완료!")
    
    return {
        "current_step": 4,
        "completed_steps": completed_steps,
        "step_outputs": step_outputs,
        "status": JobStatus.COMPLETED,
        "timestamp": datetime.now().isoformat()
    }

print("✅ Step 4 노드 구현 완료")

✅ Step 4 노드 구현 완료


# Section 2: 파일 I/O 시스템
---

## 목표
- 각 단계 결과를 JSON 파일로 저장
- 저장된 파일을 로드하는 함수 구현
- 체크포인트 복원 시 사용

## 파일 구조
runs/  
└── run_abc123/  
├── step_01.json  ← Step 1 결과  
├── step_02.json  ← Step 2 결과  
├── step_03.json  ← Step 3 결과  
├── step_04.json  ← Step 4 결과  
├── checkpoint.json  ← 체크포인트  
└── final_result.json  ← 최종 결과  

## 파일 내용 예시

### step_01.json
```json{
"step": 1,  
"step_name": "요구사항 분석",  
"timestamp": "2025-01-04T12:00:00",  
"output": {  
"analysis": "...",  
"plan": "...",  
"key_points": [...]  
}
}
```

### step_02.json
```json
{
"step": 2,  
"step_name": "정보 수집",  
"timestamp": "2025-01-04T12:01:00",  
"output": {  
"collected_data": "...",  
"sources": [...],  
"summary": "..."  
}
}
```

## 구현 함수

1. **save_step_output()**: 단계별 결과 저장
2. **load_step_output()**: 저장된 결과 로드
3. **save_final_result()**: 최종 결과물 저장

In [10]:
def save_step_output(
    run_id: str,
    step_num: int,
    output: Dict[str, Any]
) -> Path:
    """단계별 출력을 JSON 파일로 저장
    
    Args:
        run_id: 작업 ID
        step_num: 단계 번호 (1-4)
        output: 저장할 출력 데이터
        
    Returns:
        저장된 파일 경로
    """
    # run 폴더 확인/생성
    run_folder = RUNS_DIR / run_id
    run_folder.mkdir(exist_ok=True)
    
    # 파일명 생성
    filename = STEP_FILE_TEMPLATE.format(step_num)
    filepath = run_folder / filename
    
    # 저장할 데이터 구조
    step_data = {
        "step": step_num,
        "step_name": PIPELINE_STEPS[step_num],
        "timestamp": datetime.now().isoformat(),
        "output": output
    }
    
    # JSON 파일로 저장
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(step_data, f, ensure_ascii=False, indent=2)
    
    print(f"  📁 저장: {filepath}")
    
    return filepath

print("✅ save_step_output() 함수 구현 완료")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 테스트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[테스트] save_step_output()")
test_run_id = "test_" + uuid.uuid4().hex[:8]
test_output = {
    "analysis": "AI 트렌드 분석 계획",
    "plan": "4단계 실행",
    "key_points": ["LLM", "멀티모달", "에이전트"]
}

saved_path = save_step_output(test_run_id, 1, test_output)
print(f"✅ 저장 완료: {saved_path}")

# 파일 내용 확인
with open(saved_path, 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)
    print(f"\n저장된 내용:")
    print(json.dumps(loaded_data, ensure_ascii=False, indent=2)[:200] + "...")

✅ save_step_output() 함수 구현 완료

[테스트] save_step_output()
  📁 저장: runs/test_358837bc/step_01.json
✅ 저장 완료: runs/test_358837bc/step_01.json

저장된 내용:
{
  "step": 1,
  "step_name": "요구사항 분석",
  "timestamp": "2026-01-05T20:21:48.609719",
  "output": {
    "analysis": "AI 트렌드 분석 계획",
    "plan": "4단계 실행",
    "key_points": [
      "LLM",
      "멀티모달",...


In [11]:
def load_step_output(
    run_id: str,
    step_num: int
) -> Optional[Dict[str, Any]]:
    """저장된 단계 출력 로드
    
    Args:
        run_id: 작업 ID
        step_num: 단계 번호 (1-4)
        
    Returns:
        로드된 출력 데이터 또는 None
    """
    # 파일 경로
    filename = STEP_FILE_TEMPLATE.format(step_num)
    filepath = RUNS_DIR / run_id / filename
    
    # 파일 존재 확인
    if not filepath.exists():
        print(f"  ⚠️ 파일 없음: {filepath}")
        return None
    
    # JSON 파일 로드
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            step_data = json.load(f)
        
        print(f"  📂 로드: {filepath}")
        return step_data
    
    except Exception as e:
        print(f"  ❌ 로드 실패: {e}")
        return None

print("✅ load_step_output() 함수 구현 완료")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 테스트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[테스트] load_step_output()")
loaded = load_step_output(test_run_id, 1)

if loaded:
    print(f"✅ 로드 성공")
    print(f"  Step: {loaded['step']}")
    print(f"  Name: {loaded['step_name']}")
    print(f"  Output keys: {list(loaded['output'].keys())}")
else:
    print(f"❌ 로드 실패")

✅ load_step_output() 함수 구현 완료

[테스트] load_step_output()
  📂 로드: runs/test_358837bc/step_01.json
✅ 로드 성공
  Step: 1
  Name: 요구사항 분석
  Output keys: ['analysis', 'plan', 'key_points']


In [12]:
def save_final_result(
    run_id: str,
    state: PipelineState
) -> Path:
    """최종 결과물 저장
    
    Args:
        run_id: 작업 ID
        state: 최종 State
        
    Returns:
        저장된 파일 경로
    """
    # run 폴더
    run_folder = RUNS_DIR / run_id
    filepath = run_folder / FINAL_RESULT_FILE
    
    # 최종 결과 구조
    final_result = {
        "run_id": run_id,
        "user_goal": state["user_goal"],
        "completed_at": datetime.now().isoformat(),
        "status": state["status"],
        "completed_steps": state["completed_steps"],
        
        # 각 단계별 핵심 결과
        "step_summaries": {
            "step_1": state["step_outputs"].get("step_1", {}).get("analysis", ""),
            "step_2": state["step_outputs"].get("step_2", {}).get("summary", ""),
            "step_3": state["step_outputs"].get("step_3", {}).get("content", ""),
            "step_4": state["step_outputs"].get("step_4", {}).get("validation", "")
        },
        
        # 최종 콘텐츠 (Step 3)
        "final_content": state["step_outputs"].get("step_3", {}).get("content", ""),
        
        # 품질 점수 (Step 4)
        "quality_score": state["step_outputs"].get("step_4", {}).get("quality_score", 0),
        
        # 전체 step_outputs
        "full_outputs": state["step_outputs"]
    }
    
    # JSON 파일로 저장
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(final_result, f, ensure_ascii=False, indent=2)
    
    print(f"  📁 최종 결과 저장: {filepath}")
    
    return filepath

print("✅ save_final_result() 함수 구현 완료")

print("""
[save_final_result 저장 내용]
  - run_id, user_goal
  - completed_at, status
  - completed_steps
  - step_summaries (각 단계 핵심)
  - final_content (Step 3 전체)
  - quality_score (Step 4 점수)
  - full_outputs (전체 데이터)
""")

✅ save_final_result() 함수 구현 완료

[save_final_result 저장 내용]
  - run_id, user_goal
  - completed_at, status
  - completed_steps
  - step_summaries (각 단계 핵심)
  - final_content (Step 3 전체)
  - quality_score (Step 4 점수)
  - full_outputs (전체 데이터)



In [13]:
def step_1_with_save(state: PipelineState) -> PipelineState:
    """Step 1 + 파일 저장"""
    # 기존 Step 1 실행
    updated_state = step_1_node(state)
    
    # 파일 저장
    print(f"\n  [파일 저장]")
    save_step_output(
        state["run_id"],
        1,
        updated_state["step_outputs"]["step_1"]
    )
    
    return updated_state

def step_2_with_save(state: PipelineState) -> PipelineState:
    """Step 2 + 파일 저장"""
    # 기존 Step 2 실행
    updated_state = step_2_node(state)
    
    # 파일 저장
    print(f"\n  [파일 저장]")
    save_step_output(
        state["run_id"],
        2,
        updated_state["step_outputs"]["step_2"]
    )
    
    return updated_state

def step_3_with_save(state: PipelineState) -> PipelineState:
    """Step 3 + 파일 저장"""
    # 기존 Step 3 실행
    updated_state = step_3_node(state)
    
    # 파일 저장
    print(f"\n  [파일 저장]")
    save_step_output(
        state["run_id"],
        3,
        updated_state["step_outputs"]["step_3"]
    )
    
    return updated_state

def step_4_with_save(state: PipelineState) -> PipelineState:
    """Step 4 + 파일 저장"""
    # 기존 Step 4 실행
    updated_state = step_4_node(state)
    
    # 파일 저장
    print(f"\n  [파일 저장]")
    save_step_output(
        state["run_id"],
        4,
        updated_state["step_outputs"]["step_4"]
    )
    
    # 최종 결과도 저장
    print(f"\n  [최종 결과 저장]")
    
    # State 완전 복사본 생성 (updated_state 업데이트 반영)
    final_state = {**state, **updated_state}
    save_final_result(state["run_id"], final_state)
    
    return updated_state

print("✅ 파일 I/O 통합 노드 구현 완료")

print("""
[파일 저장 통합]
  - step_1_with_save: Step 1 실행 → step_01.json 저장
  - step_2_with_save: Step 2 실행 → step_02.json 저장
  - step_3_with_save: Step 3 실행 → step_03.json 저장
  - step_4_with_save: Step 4 실행 → step_04.json + final_result.json 저장

[파일 구조]
  runs/run_abc123/
  ├── step_01.json  ✅
  ├── step_02.json  ✅
  ├── step_03.json  ✅
  ├── step_04.json  ✅
  └── final_result.json  ✅
""")

✅ 파일 I/O 통합 노드 구현 완료

[파일 저장 통합]
  - step_1_with_save: Step 1 실행 → step_01.json 저장
  - step_2_with_save: Step 2 실행 → step_02.json 저장
  - step_3_with_save: Step 3 실행 → step_03.json 저장
  - step_4_with_save: Step 4 실행 → step_04.json + final_result.json 저장

[파일 구조]
  runs/run_abc123/
  ├── step_01.json  ✅
  ├── step_02.json  ✅
  ├── step_03.json  ✅
  ├── step_04.json  ✅
  └── final_result.json  ✅



# Section 3: 체크포인트 시스템 (STOP/resume)
---

## 목표
- 파이프라인 중단 기능 (STOP)
- 저장된 지점부터 재개 (resume)
- 체크포인트 파일 관리

## 체크포인트란?

게임의 **세이브 파일**처럼, 현재 진행 상황을 저장해두고   
나중에 그 지점부터 다시 시작할 수 있게 하는 기능.  
Step 1 완료 → Step 2 완료 → STOP ⏸️  
↓  
checkpoint.json 저장  
↓  
(나중에...)  
↓  
resume("run_abc123")  
↓  
Step 3부터 재개 ▶️  

## checkpoint.json 구조
```json{
"run_id": "run_abc123",
"user_goal": "AI 트렌드 보고서",
"current_step": 2,
"completed_steps": [1, 2],
"step_outputs": {
"step_1": {...},
"step_2": {...}
},
"status": "stopped",
"timestamp": "2025-01-04T12:30:00"
}
```

## 구현 함수

1. **save_checkpoint()**: 현재 상태 저장
2. **load_checkpoint()**: 저장된 상태 복원
3. **should_stop_node()**: STOP 확인 노드
4. **resume()**: 체크포인트부터 재개

## STOP 동작 방식
```python
# 사용자가 STOP 입력
state["stop_requested"] = True다음 단계로 넘어가기 전 체크
if state["stop_requested"]:
save_checkpoint(state)
return "STOP"
else:
return "CONTINUE"
```

In [14]:
def save_checkpoint(state: PipelineState) -> Path:
    """체크포인트 저장
    
    Args:
        state: 현재 파이프라인 State
        
    Returns:
        저장된 체크포인트 파일 경로
    """
    run_id = state["run_id"]
    
    # run 폴더
    run_folder = RUNS_DIR / run_id
    run_folder.mkdir(exist_ok=True)
    filepath = run_folder / CHECKPOINT_FILE
    
    # 체크포인트 데이터
    checkpoint_data = {
        "run_id": run_id,
        "user_goal": state["user_goal"],
        "current_step": state["current_step"],
        "completed_steps": state["completed_steps"],
        "step_outputs": state["step_outputs"],
        "status": JobStatus.STOPPED,  # 중단 상태
        "timestamp": datetime.now().isoformat()
    }
    
    # JSON 파일로 저장
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(checkpoint_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n  💾 체크포인트 저장: {filepath}")
    print(f"     Run ID: {run_id}")
    print(f"     현재 단계: Step {state['current_step']}")
    print(f"     완료 단계: {state['completed_steps']}")
    
    return filepath

print("✅ save_checkpoint() 함수 구현 완료")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 테스트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[테스트] save_checkpoint()")
test_state = create_initial_state("AI 트렌드 보고서")
test_state["current_step"] = 2
test_state["completed_steps"] = [1, 2]
test_state["step_outputs"] = {
    "step_1": {"analysis": "분석 완료"},
    "step_2": {"data": "데이터 수집 완료"}
}

checkpoint_path = save_checkpoint(test_state)
print(f"\n✅ 체크포인트 저장 완료: {checkpoint_path}")

✅ save_checkpoint() 함수 구현 완료

[테스트] save_checkpoint()

  💾 체크포인트 저장: runs/run_95944495/checkpoint.json
     Run ID: run_95944495
     현재 단계: Step 2
     완료 단계: [1, 2]

✅ 체크포인트 저장 완료: runs/run_95944495/checkpoint.json


In [15]:
def load_checkpoint(run_id: str) -> Optional[PipelineState]:
    """체크포인트 로드
    
    Args:
        run_id: 작업 ID
        
    Returns:
        복원된 PipelineState 또는 None
    """
    # 체크포인트 파일 경로
    filepath = RUNS_DIR / run_id / CHECKPOINT_FILE
    
    # 파일 존재 확인
    if not filepath.exists():
        print(f"  ⚠️ 체크포인트 파일 없음: {filepath}")
        return None
    
    # JSON 파일 로드
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            checkpoint_data = json.load(f)
        
        print(f"\n  💾 체크포인트 로드: {filepath}")
        print(f"     Run ID: {checkpoint_data['run_id']}")
        print(f"     요청: {checkpoint_data['user_goal']}")
        print(f"     현재 단계: Step {checkpoint_data['current_step']}")
        print(f"     완료 단계: {checkpoint_data['completed_steps']}")
        
        # PipelineState 복원
        restored_state: PipelineState = {
            "run_id": checkpoint_data["run_id"],
            "user_goal": checkpoint_data["user_goal"],
            "current_step": checkpoint_data["current_step"],
            "completed_steps": checkpoint_data["completed_steps"],
            "step_outputs": checkpoint_data["step_outputs"],
            "status": JobStatus.RUNNING,  # 재개하므로 RUNNING으로 변경
            "stop_requested": False,
            "timestamp": datetime.now().isoformat()
        }
        
        return restored_state
    
    except Exception as e:
        print(f"  ❌ 체크포인트 로드 실패: {e}")
        return None

print("✅ load_checkpoint() 함수 구현 완료")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 테스트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[테스트] load_checkpoint()")
restored = load_checkpoint(test_state["run_id"])

if restored:
    print(f"\n✅ 체크포인트 로드 성공")
    print(f"  다음 단계: Step {restored['current_step'] + 1}")
    print(f"  상태: {restored['status']}")
else:
    print(f"\n❌ 체크포인트 로드 실패")

✅ load_checkpoint() 함수 구현 완료

[테스트] load_checkpoint()

  💾 체크포인트 로드: runs/run_95944495/checkpoint.json
     Run ID: run_95944495
     요청: AI 트렌드 보고서
     현재 단계: Step 2
     완료 단계: [1, 2]

✅ 체크포인트 로드 성공
  다음 단계: Step 3
  상태: running


In [16]:
def should_stop_check(state: PipelineState) -> str:
    """STOP 확인 노드 (조건부 엣지용)
    
    Args:
        state: 현재 파이프라인 State
        
    Returns:
        "stop" | "continue_to_step_X"
    """
    # STOP 요청 확인
    if state.get("stop_requested", False):
        print(f"\n  ⏸️ STOP 요청 감지")
        print(f"     현재 단계: Step {state['current_step']}")
        return "stop"
    
    # 다음 단계로 계속
    current = state["current_step"]
    
    if current == 0:
        return "continue_to_step_1"
    elif current == 1:
        return "continue_to_step_2"
    elif current == 2:
        return "continue_to_step_3"
    elif current == 3:
        return "continue_to_step_4"
    elif current >= 4:
        return "end"
    else:
        return "continue_to_step_1"

print("✅ should_stop_check 노드 구현 완료")

print("""
[should_stop_check 동작]

1. stop_requested == True?
   → "stop" 반환
   → save_checkpoint_node로 이동

2. stop_requested == False?
   → current_step에 따라 다음 단계로
   → "continue_to_step_X" 반환

[조건부 엣지 매핑]
  "stop" → save_checkpoint_node
  "continue_to_step_1" → step_1_with_save
  "continue_to_step_2" → step_2_with_save
  "continue_to_step_3" → step_3_with_save
  "continue_to_step_4" → step_4_with_save
  "end" → END
""")


✅ should_stop_check 노드 구현 완료

[should_stop_check 동작]

1. stop_requested == True?
   → "stop" 반환
   → save_checkpoint_node로 이동

2. stop_requested == False?
   → current_step에 따라 다음 단계로
   → "continue_to_step_X" 반환

[조건부 엣지 매핑]
  "stop" → save_checkpoint_node
  "continue_to_step_1" → step_1_with_save
  "continue_to_step_2" → step_2_with_save
  "continue_to_step_3" → step_3_with_save
  "continue_to_step_4" → step_4_with_save
  "end" → END



In [17]:
def save_checkpoint_node(state: PipelineState) -> PipelineState:
    """체크포인트 저장 노드
    
    Args:
        state: 현재 파이프라인 State
        
    Returns:
        업데이트된 State (status: stopped)
    """
    print("\n" + "="*80)
    print("[Save Checkpoint] 체크포인트 저장 중...")
    print("="*80)
    
    # 체크포인트 저장
    checkpoint_path = save_checkpoint(state)
    
    print(f"\n  ✅ 체크포인트 저장 완료")
    print(f"  📁 경로: {checkpoint_path}")
    print(f"\n  재개 방법:")
    print(f"     resume('{state['run_id']}')")
    
    return {
        "status": JobStatus.STOPPED
    }

print("✅ save_checkpoint_node 구현 완료")

✅ save_checkpoint_node 구현 완료


In [18]:
def resume(run_id: str):
    """체크포인트부터 파이프라인 재개
    
    Args:
        run_id: 재개할 작업 ID
    """
    print(f"\n[Resume] 작업 재개: {run_id}")
    
    # 1. 체크포인트 로드
    state = load_checkpoint(run_id)
    
    if not state:
        print(f"  ❌ 체크포인트를 찾을 수 없습니다.")
        return None
    
    print(f"\n  ✅ 체크포인트 로드 성공")
    print(f"     Step {state['current_step'] + 1}부터 재개")
    
    # 2. 그래프 실행 (나중에 구현)
    # result = pipeline_graph.invoke(state)
    # return result
    
    print(f"\n  ⚠️ resume() 함수는 그래프 구성 후 완성됩니다")
    return state

print("✅ resume() 함수 시그니처 정의 완료")

print("""
[resume() 동작 흐름]

1. load_checkpoint(run_id)
   → 저장된 상태 복원

2. 복원된 state로 그래프 실행
   → current_step 다음부터 시작
   → 예: current_step=2 → Step 3부터

3. 나머지 단계 완료
   → Step 3 → Step 4 → 완료

[예시]
  # Step 2 완료 후 중단
  save_checkpoint(state)
  
  # 나중에 재개
  resume("run_abc123")
  → Step 3부터 실행
""")

✅ resume() 함수 시그니처 정의 완료

[resume() 동작 흐름]

1. load_checkpoint(run_id)
   → 저장된 상태 복원

2. 복원된 state로 그래프 실행
   → current_step 다음부터 시작
   → 예: current_step=2 → Step 3부터

3. 나머지 단계 완료
   → Step 3 → Step 4 → 완료

[예시]
  # Step 2 완료 후 중단
  save_checkpoint(state)

  # 나중에 재개
  resume("run_abc123")
  → Step 3부터 실행



# Section 4: 그래프 구성 및 대기열
---

## 목표
- StateGraph로 전체 파이프라인 연결
- 조건부 엣지로 STOP/continue 분기
- 대기열 시스템 구현
- resume() 함수 완성

## 그래프 구조START
↓  
should_stop_check (조건부)  
├─ stop → save_checkpoint_node → END  
├─ continue_to_step_1 → step_1_with_save → should_stop_check  
├─ continue_to_step_2 → step_2_with_save → should_stop_check  
├─ continue_to_step_3 → step_3_with_save → should_stop_check  
├─ continue_to_step_4 → step_4_with_save → END  
└─ end → END  

## 조건부 엣지

각 Step 완료 후 should_stop_check로 돌아와서:
- stop_requested == True → 체크포인트 저장 후 종료
- stop_requested == False → 다음 Step으로

## 대기열 시스템
```python
job_queue = Queue()  # FIFO 큐
def add_job(user_goal):
    job_queue.put(user_goal)
    process_queue() # 순차 처리
def process_queue():
    if not job_queue.empty():
        job = job_queue.get()
        run_pipeline(job)
```

## 동시 요청 처리
Request 1: "AI 트렌드" → 실행 중 ⚙️  
Request 2: "블록체인" → 대기 중 ⏸️Request 1 완료 후  
Request 2: "블록체인" → 실행 중 ⚙️  

In [19]:
print("\n[1/4] StateGraph 생성 중...")
pipeline_graph_builder = StateGraph(PipelineState)
print("  ✅ StateGraph 생성 완료")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 노드 추가
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[2/4] 노드 추가 중...")

# Step 노드들 (파일 저장 포함)
pipeline_graph_builder.add_node("step_1", step_1_with_save)
print("  ✅ 'step_1' 노드 추가")

pipeline_graph_builder.add_node("step_2", step_2_with_save)
print("  ✅ 'step_2' 노드 추가")

pipeline_graph_builder.add_node("step_3", step_3_with_save)
print("  ✅ 'step_3' 노드 추가")

pipeline_graph_builder.add_node("step_4", step_4_with_save)
print("  ✅ 'step_4' 노드 추가")

# 체크포인트 노드
pipeline_graph_builder.add_node("save_checkpoint", save_checkpoint_node)
print("  ✅ 'save_checkpoint' 노드 추가")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 엣지 추가
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[3/4] 엣지 추가 중...")

# START → 조건부 엣지 (should_stop_check)
pipeline_graph_builder.add_conditional_edges(
    START,
    should_stop_check,
    {
        "stop": "save_checkpoint",
        "continue_to_step_1": "step_1",
        "continue_to_step_2": "step_2",
        "continue_to_step_3": "step_3",
        "continue_to_step_4": "step_4",
        "end": END
    }
)
print("  ✅ START → 조건부 엣지")

# 각 Step 완료 후 → 조건부 엣지 (루프)
pipeline_graph_builder.add_conditional_edges(
    "step_1",
    should_stop_check,
    {
        "stop": "save_checkpoint",
        "continue_to_step_2": "step_2",
        "end": END
    }
)
print("  ✅ step_1 → 조건부 엣지")

pipeline_graph_builder.add_conditional_edges(
    "step_2",
    should_stop_check,
    {
        "stop": "save_checkpoint",
        "continue_to_step_3": "step_3",
        "end": END
    }
)
print("  ✅ step_2 → 조건부 엣지")

pipeline_graph_builder.add_conditional_edges(
    "step_3",
    should_stop_check,
    {
        "stop": "save_checkpoint",
        "continue_to_step_4": "step_4",
        "end": END
    }
)
print("  ✅ step_3 → 조건부 엣지")

# Step 4 완료 → END
pipeline_graph_builder.add_edge("step_4", END)
print("  ✅ step_4 → END")

# save_checkpoint → END
pipeline_graph_builder.add_edge("save_checkpoint", END)
print("  ✅ save_checkpoint → END")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 컴파일
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[4/4] 그래프 컴파일 중...")

# MemorySaver (체크포인터)
memory = MemorySaver()

# 컴파일
pipeline_graph = pipeline_graph_builder.compile(checkpointer=memory)

print("  ✅ 그래프 컴파일 완료")

print("\n" + "="*80)
print("✅ StateGraph 구성 완료")
print("="*80)

print("""
[그래프 구조]

  START
    ↓ (조건부)
  should_stop_check
    ├─ stop? → save_checkpoint → END
    ├─ step 0? → step_1 → (조건부) → ...
    ├─ step 1? → step_2 → (조건부) → ...
    ├─ step 2? → step_3 → (조건부) → ...
    ├─ step 3? → step_4 → END
    └─ step 4? → END

[조건부 엣지 동작]
  1. 각 Step 완료 후 should_stop_check 실행
  2. stop_requested == True → save_checkpoint
  3. stop_requested == False → 다음 Step
  
[STOP/resume 흐름]
  Step 2 완료 → stop_requested=True 
  → save_checkpoint → checkpoint.json 저장
  
  나중에...
  resume("run_abc123")
  → checkpoint 로드 → Step 3부터 재개
""")


[1/4] StateGraph 생성 중...
  ✅ StateGraph 생성 완료

[2/4] 노드 추가 중...
  ✅ 'step_1' 노드 추가
  ✅ 'step_2' 노드 추가
  ✅ 'step_3' 노드 추가
  ✅ 'step_4' 노드 추가
  ✅ 'save_checkpoint' 노드 추가

[3/4] 엣지 추가 중...
  ✅ START → 조건부 엣지
  ✅ step_1 → 조건부 엣지
  ✅ step_2 → 조건부 엣지
  ✅ step_3 → 조건부 엣지
  ✅ step_4 → END
  ✅ save_checkpoint → END

[4/4] 그래프 컴파일 중...
  ✅ 그래프 컴파일 완료

✅ StateGraph 구성 완료

[그래프 구조]

  START
    ↓ (조건부)
  should_stop_check
    ├─ stop? → save_checkpoint → END
    ├─ step 0? → step_1 → (조건부) → ...
    ├─ step 1? → step_2 → (조건부) → ...
    ├─ step 2? → step_3 → (조건부) → ...
    ├─ step 3? → step_4 → END
    └─ step 4? → END

[조건부 엣지 동작]
  1. 각 Step 완료 후 should_stop_check 실행
  2. stop_requested == True → save_checkpoint
  3. stop_requested == False → 다음 Step

[STOP/resume 흐름]
  Step 2 완료 → stop_requested=True 
  → save_checkpoint → checkpoint.json 저장

  나중에...
  resume("run_abc123")
  → checkpoint 로드 → Step 3부터 재개



In [20]:
def resume(run_id: str) -> Optional[PipelineState]:
    """체크포인트부터 파이프라인 재개
    
    Args:
        run_id: 재개할 작업 ID
        
    Returns:
        최종 PipelineState 또는 None
    """
    print("\n" + "="*80)
    print(f"[Resume] 작업 재개: {run_id}")
    print("="*80)
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 1. 체크포인트 로드
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    state = load_checkpoint(run_id)
    
    if not state:
        print(f"  ❌ 체크포인트를 찾을 수 없습니다.")
        return None
    
    print(f"\n  ✅ 체크포인트 로드 성공")
    print(f"     현재 단계: Step {state['current_step']}")
    print(f"     다음 실행: Step {state['current_step'] + 1}")
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 2. 그래프 실행 (재개)
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    print(f"\n  ▶️ 파이프라인 재개 중...")
    
    # 설정 (thread_id)
    config = {"configurable": {"thread_id": run_id}}
    
    # 그래프 실행
    result = pipeline_graph.invoke(state, config=config)
    
    print(f"\n  ✅ 파이프라인 재개 완료")
    print(f"     최종 상태: {result['status']}")
    
    return result

print("✅ resume() 함수 완성")

print("""
[resume() 동작 흐름]

1. load_checkpoint(run_id)
   → checkpoint.json 파일 로드
   → PipelineState 복원

2. pipeline_graph.invoke(state)
   → 복원된 state로 그래프 실행
   → current_step에 따라 다음 단계부터 시작
   
3. 나머지 단계 완료
   → Step 3 → Step 4 → 완료

[예시]
  # Step 2 완료 후 중단
  state["stop_requested"] = True
  → checkpoint.json 저장
  
  # 나중에 재개
  resume("run_abc123")
  → Step 3 실행 → Step 4 실행 → 완료
""")

✅ resume() 함수 완성

[resume() 동작 흐름]

1. load_checkpoint(run_id)
   → checkpoint.json 파일 로드
   → PipelineState 복원

2. pipeline_graph.invoke(state)
   → 복원된 state로 그래프 실행
   → current_step에 따라 다음 단계부터 시작

3. 나머지 단계 완료
   → Step 3 → Step 4 → 완료

[예시]
  # Step 2 완료 후 중단
  state["stop_requested"] = True
  → checkpoint.json 저장

  # 나중에 재개
  resume("run_abc123")
  → Step 3 실행 → Step 4 실행 → 완료



In [21]:
job_queue = Queue()
queue_lock = Lock()
currently_running = None  # 현재 실행 중인 run_id

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# run_pipeline() 함수
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def run_pipeline(user_goal: str, stop_at_step: Optional[int] = None) -> PipelineState:
    """파이프라인 실행
    
    Args:
        user_goal: 사용자 요청
        stop_at_step: 특정 Step에서 중단 (테스트용)
        
    Returns:
        최종 PipelineState
    """
    global currently_running
    
    # 초기 State 생성
    state = create_initial_state(user_goal)
    run_id = state["run_id"]
    
    # run 폴더 생성
    create_run_folder(run_id)
    
    print("\n" + "="*80)
    print(f"[Pipeline Start] {run_id}")
    print("="*80)
    print(f"요청: {user_goal}")
    
    # 현재 실행 중 표시
    currently_running = run_id
    
    # 설정
    config = {"configurable": {"thread_id": run_id}}
    
    # stop_at_step 설정 시 중간에 중단
    if stop_at_step:
        print(f"\n⚠️ 테스트 모드: Step {stop_at_step}에서 중단 예정")
    
    # 그래프 실행
    result = pipeline_graph.invoke(state, config=config)
    
    # stop_at_step 처리
    if stop_at_step and result["current_step"] == stop_at_step:
        print(f"\n⏸️ Step {stop_at_step} 완료 후 중단")
        result["stop_requested"] = True
        save_checkpoint(result)
    
    # 실행 완료
    currently_running = None
    
    print("\n" + "="*80)
    print(f"[Pipeline End] {run_id}")
    print("="*80)
    print(f"최종 상태: {result['status']}")
    
    return result

print("✅ run_pipeline() 함수 구현 완료")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 대기열 추가 함수
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def add_job(user_goal: str):
    """작업을 대기열에 추가
    
    Args:
        user_goal: 사용자 요청
    """
    print(f"\n[Queue] 작업 추가: '{user_goal}'")
    
    # 대기열에 추가
    job_queue.put(user_goal)
    
    # 큐 상태 출력
    queue_size = job_queue.qsize()
    print(f"  대기열 크기: {queue_size}")
    
    if currently_running:
        print(f"  ⚙️ 실행 중: {currently_running}")
        print(f"  ⏸️ 대기 중: {queue_size}개")
    else:
        print(f"  ✅ 즉시 실행 가능")
        process_queue()

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 대기열 처리 함수
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def process_queue():
    """대기열 순차 처리"""
    global currently_running
    
    with queue_lock:
        # 이미 실행 중이면 패스
        if currently_running:
            return
        
        # 큐가 비어있으면 패스
        if job_queue.empty():
            print("\n[Queue] 대기열이 비어있습니다.")
            return
        
        # 다음 작업 가져오기
        user_goal = job_queue.get()
        
        print(f"\n[Queue] 다음 작업 실행 중...")
        print(f"  요청: '{user_goal}'")
        print(f"  남은 대기: {job_queue.qsize()}개")
        
        # 파이프라인 실행
        try:
            run_pipeline(user_goal)
        except Exception as e:
            print(f"\n❌ 파이프라인 실행 실패: {e}")
        finally:
            currently_running = None
            
            # 다음 작업 처리
            if not job_queue.empty():
                print(f"\n[Queue] 다음 작업으로 이동...")
                process_queue()

print("✅ 대기열 시스템 구현 완료")

print("""
[대기열 시스템 동작]

1. add_job(user_goal)
   → 대기열에 추가
   → 즉시 실행 가능하면 process_queue() 호출

2. process_queue()
   → 큐에서 작업 하나 가져오기
   → run_pipeline() 실행
   → 완료 후 다음 작업 처리

3. 동시 요청 처리
   Request 1: 즉시 실행 ⚙️
   Request 2: 대기열 추가 ⏸️
   Request 1 완료 → Request 2 실행 ⚙️

[FIFO (선입선출)]
  먼저 들어온 요청이 먼저 처리됨
""")

✅ run_pipeline() 함수 구현 완료
✅ 대기열 시스템 구현 완료

[대기열 시스템 동작]

1. add_job(user_goal)
   → 대기열에 추가
   → 즉시 실행 가능하면 process_queue() 호출

2. process_queue()
   → 큐에서 작업 하나 가져오기
   → run_pipeline() 실행
   → 완료 후 다음 작업 처리

3. 동시 요청 처리
   Request 1: 즉시 실행 ⚙️
   Request 2: 대기열 추가 ⏸️
   Request 1 완료 → Request 2 실행 ⚙️

[FIFO (선입선출)]
  먼저 들어온 요청이 먼저 처리됨



# Section 5: 시나리오 테스트
---

## 목표
- 3개 핵심 시나리오로 전체 시스템 검증
- 각 기능의 정상 동작 확인
- 파일 저장 및 복원 검증

## 3개 시나리오 (과제 요구사항)

### 시나리오 A: 정상 실행
**목적**: 4단계 파이프라인 정상 동작 확인  
  
실행:  
1. 사용자 요청 입력
2. Step 1 → Step 2 → Step 3 → Step 4 순차 실행
3. 각 단계마다 JSON 파일 저장
  
검증:  
- ✅ step_01.json, step_02.json, step_03.json, step_04.json 생성
- ✅ final_result.json 생성
- ✅ status: "completed"
- ✅ 모든 step_outputs 존재

---

### 시나리오 B: STOP 후 재개
**목적**: 체크포인트 저장/복원 기능 확인

실행:
1. 파이프라인 시작
2. Step 2 완료 후 중단 (STOP)
3. checkpoint.json 저장 확인
4. resume(run_id) 호출
5. Step 3 → Step 4 이어서 완료

검증:
- ✅ Step 2까지 step_01.json, step_02.json 생성
- ✅ checkpoint.json 생성 (current_step: 2)
- ✅ resume 후 step_03.json, step_04.json 생성
- ✅ final_result.json 생성
- ✅ 데이터 연속성 확인 (Step 1, 2 데이터 유지)

---

### 시나리오 C: 대기열 테스트
**목적**: 동시 요청 순차 처리 확인

실행:
1. 요청 1 추가: "AI 윤리 에세이"
2. 요청 2 추가: "블록체인 기술 분석" (거의 동시)
3. 요청 1이 실행되는 동안 요청 2는 대기
4. 요청 1 완료 후 요청 2 자동 실행

검증:
- ✅ 요청 1 실행 중 currently_running 표시
- ✅ 요청 2 대기열 추가 확인
- ✅ 요청 1 완료 후 요청 2 자동 시작
- ✅ 각 요청별 별도 폴더 생성 (run_xxx, run_yyy)
- ✅ 두 작업 모두 정상 완료

---

## 시나리오별 예상 파일 구조

### 시나리오 A
runs/run_abc123/  
├── step_01.json  
├── step_02.json  
├── step_03.json  
├── step_04.json  
└── final_result.json  

### 시나리오 B
runs/run_def456/  
├── step_01.json       (중단 전)  
├── step_02.json       (중단 전)  
├── checkpoint.json    (중단 시 저장)  
├── step_03.json       (재개 후)  
├── step_04.json       (재개 후)  
└── final_result.json  (재개 후)  

### 시나리오 C
runs/run_ghi789/       (요청 1)  
├── step_01.json  
├── step_02.json  
├── step_03.json  
├── step_04.json  
└── final_result.jsonruns/run_jkl012/       (요청 2)  
├── step_01.json  
├── step_02.json  
├── step_03.json  
├── step_04.json  
└── final_result.json  

In [22]:
def verify_files(run_id: str, expected_files: List[str]) -> Dict[str, bool]:
    """파일 존재 여부 검증
    
    Args:
        run_id: 작업 ID
        expected_files: 예상 파일 리스트
        
    Returns:
        파일별 존재 여부 딕셔너리
    """
    run_folder = RUNS_DIR / run_id
    results = {}
    
    print(f"\n  [파일 검증] {run_id}")
    
    for filename in expected_files:
        filepath = run_folder / filename
        exists = filepath.exists()
        results[filename] = exists
        
        status = "✅" if exists else "❌"
        print(f"    {status} {filename}")
    
    return results

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# State 검증 함수
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def verify_state(state: PipelineState, expected_status: str, expected_steps: List[int]):
    """State 검증
    
    Args:
        state: 검증할 State
        expected_status: 예상 상태
        expected_steps: 예상 완료 단계
    """
    print(f"\n  [State 검증]")
    
    # 상태 확인
    status_ok = state["status"] == expected_status
    status_icon = "✅" if status_ok else "❌"
    print(f"    {status_icon} status: {state['status']} (예상: {expected_status})")
    
    # 완료 단계 확인
    steps_ok = state["completed_steps"] == expected_steps
    steps_icon = "✅" if steps_ok else "❌"
    print(f"    {steps_icon} completed_steps: {state['completed_steps']} (예상: {expected_steps})")
    
    # step_outputs 확인
    outputs_count = len(state["step_outputs"])
    expected_count = len(expected_steps)
    outputs_ok = outputs_count == expected_count
    outputs_icon = "✅" if outputs_ok else "❌"
    print(f"    {outputs_icon} step_outputs: {outputs_count}개 (예상: {expected_count}개)")
    
    return status_ok and steps_ok and outputs_ok

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 시나리오 요약 출력
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def print_scenario_summary(scenario_name: str, run_id: str, state: PipelineState):
    """시나리오 결과 요약
    
    Args:
        scenario_name: 시나리오 이름
        run_id: 작업 ID
        state: 최종 State
    """
    print("\n" + "="*80)
    print(f"[{scenario_name}] 결과 요약")
    print("="*80)
    
    print(f"\n  Run ID: {run_id}")
    print(f"  요청: {state['user_goal']}")
    print(f"  최종 상태: {state['status']}")
    print(f"  완료 단계: {state['completed_steps']}")
    print(f"  현재 단계: Step {state['current_step']}")
    
    # 품질 점수 (Step 4가 있으면)
    if "step_4" in state["step_outputs"]:
        quality = state["step_outputs"]["step_4"].get("quality_score", 0)
        print(f"  품질 점수: {quality}/10")

print("✅ 시나리오 헬퍼 함수 정의 완료")

✅ 시나리오 헬퍼 함수 정의 완료


In [23]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 실행
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

scenario_a_goal = "AI 윤리에 대한 300자 에세이 작성"

print(f"\n요청: '{scenario_a_goal}'")
print(f"\n{'='*80}")

# 파이프라인 실행
scenario_a_result = run_pipeline(scenario_a_goal)

scenario_a_run_id = scenario_a_result["run_id"]

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 검증
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "="*80)
print("[시나리오 A] 검증")
print("="*80)

# 파일 검증
expected_files = [
    "step_01.json",
    "step_02.json", 
    "step_03.json",
    "step_04.json",
    "final_result.json"
]

file_results = verify_files(scenario_a_run_id, expected_files)

# State 검증
state_ok = verify_state(
    scenario_a_result,
    expected_status=JobStatus.COMPLETED,
    expected_steps=[1, 2, 3, 4]
)

# 전체 성공 여부
all_files_ok = all(file_results.values())
scenario_a_success = all_files_ok and state_ok

# 요약
print_scenario_summary("시나리오 A", scenario_a_run_id, scenario_a_result)

print(f"\n{'='*80}")
if scenario_a_success:
    print("✅ 시나리오 A: 정상 실행 - 성공")
else:
    print("❌ 시나리오 A: 정상 실행 - 실패")
print("="*80)


요청: 'AI 윤리에 대한 300자 에세이 작성'

  ✅ Run 폴더 생성: runs/run_0a44e946

[Pipeline Start] run_0a44e946
요청: AI 윤리에 대한 300자 에세이 작성

[Step 1] 요구사항 분석
Run ID: run_0a44e946
요청: AI 윤리에 대한 300자 에세이 작성

  [1/2] 요구사항 분석 중...
  ✅ 분석 완료
     핵심 포인트: 5개

  [2/2] State 업데이트 중...
  ✅ Step 1 완료

  [파일 저장]
  📁 저장: runs/run_0a44e946/step_01.json

[Step 2] 정보 수집

  [1/3] 웹 검색 중...
     검색어: 'AI 윤리에 대한 300자 에세이 작성'
     ✅ 검색 완료: 5개 결과

  [2/3] 정보 정리 중...
  ✅ 정리 완료
     출처: 5개

  [3/3] State 업데이트 중...
  ✅ Step 2 완료

  [파일 저장]
  📁 저장: runs/run_0a44e946/step_02.json

[Step 3] 콘텐츠 생성
  참고: Step 1 분석 + Step 2 정보

  [1/2] 콘텐츠 생성 중...
  ✅ 생성 완료
     단어 수: 300개

  [2/2] State 업데이트 중...
  ✅ Step 3 완료

  [파일 저장]
  📁 저장: runs/run_0a44e946/step_03.json

[Step 4] 최종 검증
  참고: Step 3 콘텐츠

  [1/2] 품질 검증 중...
  ✅ 검증 완료
     품질 점수: 8.5/10
     상태: approved

  [2/2] State 업데이트 중...
  ✅ Step 4 완료
  ✅ 전체 파이프라인 완료!

  [파일 저장]
  📁 저장: runs/run_0a44e946/step_04.json

  [최종 결과 저장]
  📁 최종 결과 저장: runs/run_0a44e946/final_result.json

[Pipel

In [24]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Phase 1: Step 2까지 실행 후 중단
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

scenario_b_goal = "블록체인 기술의 미래 전망"

print(f"\n[Phase 1] Step 2까지 실행 후 중단")
print(f"요청: '{scenario_b_goal}'")
print(f"\n{'='*80}")

# Step 2에서 중단하도록 실행
scenario_b_phase1 = run_pipeline(scenario_b_goal, stop_at_step=2)

scenario_b_run_id = scenario_b_phase1["run_id"]

print(f"\n⏸️ Step 2 완료 후 중단됨")

# Phase 1 파일 검증
print("\n[Phase 1 검증]")
phase1_files = [
    "step_01.json",
    "step_02.json",
    "checkpoint.json"
]

phase1_file_results = verify_files(scenario_b_run_id, phase1_files)

# checkpoint.json 내용 확인
checkpoint_path = RUNS_DIR / scenario_b_run_id / CHECKPOINT_FILE
if checkpoint_path.exists():
    with open(checkpoint_path, 'r', encoding='utf-8') as f:
        checkpoint_data = json.load(f)
    
    print(f"\n  [checkpoint.json 내용]")
    print(f"    current_step: {checkpoint_data['current_step']}")
    print(f"    completed_steps: {checkpoint_data['completed_steps']}")
    print(f"    status: {checkpoint_data['status']}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Phase 2: resume로 재개
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print(f"\n{'='*80}")
print(f"[Phase 2] Resume - Step 3부터 재개")
print(f"{'='*80}")

import time
time.sleep(1)  # 잠시 대기 (로그 구분)

# resume 실행
scenario_b_phase2 = resume(scenario_b_run_id)

if scenario_b_phase2:
    print(f"\n✅ Resume 성공")
    
    # Phase 2 파일 검증
    print("\n[Phase 2 검증]")
    phase2_files = [
        "step_01.json",
        "step_02.json",
        "step_03.json",
        "step_04.json",
        "checkpoint.json",
        "final_result.json"
    ]
    
    phase2_file_results = verify_files(scenario_b_run_id, phase2_files)
    
    # State 검증
    state_ok = verify_state(
        scenario_b_phase2,
        expected_status=JobStatus.COMPLETED,
        expected_steps=[1, 2, 3, 4]
    )
    
    # 데이터 연속성 확인
    print(f"\n  [데이터 연속성 확인]")
    step1_data = scenario_b_phase2["step_outputs"].get("step_1")
    step2_data = scenario_b_phase2["step_outputs"].get("step_2")
    
    step1_ok = step1_data is not None
    step2_ok = step2_data is not None
    
    step1_icon = "✅" if step1_ok else "❌"
    step2_icon = "✅" if step2_ok else "❌"
    
    print(f"    {step1_icon} Step 1 데이터 유지")
    print(f"    {step2_icon} Step 2 데이터 유지")
    
    # 전체 성공 여부
    all_files_ok = all(phase2_file_results.values())
    data_continuity_ok = step1_ok and step2_ok
    scenario_b_success = all_files_ok and state_ok and data_continuity_ok
    
    # 요약
    print_scenario_summary("시나리오 B", scenario_b_run_id, scenario_b_phase2)
    
else:
    print(f"\n❌ Resume 실패")
    scenario_b_success = False

print(f"\n{'='*80}")
if scenario_b_success:
    print("✅ 시나리오 B: STOP 후 재개 - 성공")
else:
    print("❌ 시나리오 B: STOP 후 재개 - 실패")
print("="*80)


[Phase 1] Step 2까지 실행 후 중단
요청: '블록체인 기술의 미래 전망'

  ✅ Run 폴더 생성: runs/run_d89df0fd

[Pipeline Start] run_d89df0fd
요청: 블록체인 기술의 미래 전망

⚠️ 테스트 모드: Step 2에서 중단 예정

[Step 1] 요구사항 분석
Run ID: run_d89df0fd
요청: 블록체인 기술의 미래 전망

  [1/2] 요구사항 분석 중...
  ✅ 분석 완료
     핵심 포인트: 5개

  [2/2] State 업데이트 중...
  ✅ Step 1 완료

  [파일 저장]
  📁 저장: runs/run_d89df0fd/step_01.json

[Step 2] 정보 수집

  [1/3] 웹 검색 중...
     검색어: '블록체인 기술의 미래 전망'
     ✅ 검색 완료: 5개 결과

  [2/3] 정보 정리 중...
  ✅ 정리 완료
     출처: 5개

  [3/3] State 업데이트 중...
  ✅ Step 2 완료

  [파일 저장]
  📁 저장: runs/run_d89df0fd/step_02.json

[Step 3] 콘텐츠 생성
  참고: Step 1 분석 + Step 2 정보

  [1/2] 콘텐츠 생성 중...
  ✅ 생성 완료
     단어 수: 487개

  [2/2] State 업데이트 중...
  ✅ Step 3 완료

  [파일 저장]
  📁 저장: runs/run_d89df0fd/step_03.json

[Step 4] 최종 검증
  참고: Step 3 콘텐츠

  [1/2] 품질 검증 중...
  ✅ 검증 완료
     품질 점수: 8.5/10
     상태: approved

  [2/2] State 업데이트 중...
  ✅ Step 4 완료
  ✅ 전체 파이프라인 완료!

  [파일 저장]
  📁 저장: runs/run_d89df0fd/step_04.json

  [최종 결과 저장]
  📁 최종 결과 저장: runs/run_d89df0fd

In [25]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 요청 추가
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

scenario_c_goal_1 = "기후변화 대응 방안"
scenario_c_goal_2 = "양자컴퓨팅 기술 개요"

print(f"\n{'='*80}")
print(f"[요청 추가]")
print(f"{'='*80}")

# 요청 1 추가 (즉시 실행될 것)
print(f"\n요청 1 추가...")
add_job(scenario_c_goal_1)

# 잠시 대기 (요청 1이 실행 중임을 확인)
import time
time.sleep(0.5)

# 요청 2 추가 (대기열에 들어갈 것)
print(f"\n요청 2 추가...")
add_job(scenario_c_goal_2)

print(f"\n현재 상태:")
print(f"  실행 중: {currently_running}")
print(f"  대기열: {job_queue.qsize()}개")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 대기 (비동기 처리를 위해 실제로는 동기적으로 처리됨)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print(f"\n⏳ 두 작업 완료 대기 중...")
print(f"   (실제로는 순차 실행되므로 시간이 걸립니다)")

# 참고: 실제 구현에서는 add_job()이 process_queue()를 호출하여
# 순차적으로 실행되므로, 이 시점에서 두 작업이 모두 완료되어 있음

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 검증: runs 폴더에서 최근 2개 run 찾기
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print(f"\n{'='*80}")
print(f"[검증]")
print(f"{'='*80}")

# runs 폴더에서 모든 run 찾기
all_runs = sorted(RUNS_DIR.glob("run_*"), key=lambda p: p.stat().st_mtime, reverse=True)

# 최근 2개 (시나리오 C의 것들)
if len(all_runs) >= 2:
    scenario_c_run_1 = all_runs[1].name  # 두 번째로 최근 (요청 1)
    scenario_c_run_2 = all_runs[0].name  # 가장 최근 (요청 2)
    
    print(f"\n최근 실행된 2개 작업:")
    print(f"  1. {scenario_c_run_1}")
    print(f"  2. {scenario_c_run_2}")
    
    # 각 작업의 파일 검증
    expected_files = [
        "step_01.json",
        "step_02.json",
        "step_03.json",
        "step_04.json",
        "final_result.json"
    ]
    
    print(f"\n[작업 1 파일 검증]")
    run1_files = verify_files(scenario_c_run_1, expected_files)
    
    print(f"\n[작업 2 파일 검증]")
    run2_files = verify_files(scenario_c_run_2, expected_files)
    
    # 성공 여부
    run1_ok = all(run1_files.values())
    run2_ok = all(run2_files.values())
    scenario_c_success = run1_ok and run2_ok
    
    # 각 작업의 최종 결과 확인
    for run_id in [scenario_c_run_1, scenario_c_run_2]:
        final_path = RUNS_DIR / run_id / "final_result.json"
        if final_path.exists():
            with open(final_path, 'r', encoding='utf-8') as f:
                final_data = json.load(f)
            
            print(f"\n[{run_id} 결과]")
            print(f"  요청: {final_data['user_goal']}")
            print(f"  상태: {final_data['status']}")
            print(f"  완료 단계: {final_data['completed_steps']}")
    
else:
    print(f"\n⚠️ 최근 실행된 작업이 2개 미만입니다.")
    scenario_c_success = False

print(f"\n{'='*80}")
if scenario_c_success:
    print("✅ 시나리오 C: 대기열 테스트 - 성공")
else:
    print("❌ 시나리오 C: 대기열 테스트 - 실패")
print("="*80)

print("""
[대기열 동작 확인]
  - 요청 1이 먼저 실행됨
  - 요청 2가 대기열에 추가됨
  - 요청 1 완료 후 요청 2가 자동으로 실행됨
  - 각 요청별로 별도의 run 폴더 생성
  - FIFO (선입선출) 방식으로 순차 처리
""")


[요청 추가]

요청 1 추가...

[Queue] 작업 추가: '기후변화 대응 방안'
  대기열 크기: 1
  ✅ 즉시 실행 가능

[Queue] 다음 작업 실행 중...
  요청: '기후변화 대응 방안'
  남은 대기: 0개
  ✅ Run 폴더 생성: runs/run_c6a2a43f

[Pipeline Start] run_c6a2a43f
요청: 기후변화 대응 방안

[Step 1] 요구사항 분석
Run ID: run_c6a2a43f
요청: 기후변화 대응 방안

  [1/2] 요구사항 분석 중...
  ✅ 분석 완료
     핵심 포인트: 5개

  [2/2] State 업데이트 중...
  ✅ Step 1 완료

  [파일 저장]
  📁 저장: runs/run_c6a2a43f/step_01.json

[Step 2] 정보 수집

  [1/3] 웹 검색 중...
     검색어: '기후변화 대응 방안'
     ✅ 검색 완료: 5개 결과

  [2/3] 정보 정리 중...
  ✅ 정리 완료
     출처: 5개

  [3/3] State 업데이트 중...
  ✅ Step 2 완료

  [파일 저장]
  📁 저장: runs/run_c6a2a43f/step_02.json

[Step 3] 콘텐츠 생성
  참고: Step 1 분석 + Step 2 정보

  [1/2] 콘텐츠 생성 중...
  ✅ 생성 완료
     단어 수: 486개

  [2/2] State 업데이트 중...
  ✅ Step 3 완료

  [파일 저장]
  📁 저장: runs/run_c6a2a43f/step_03.json

[Step 4] 최종 검증
  참고: Step 3 콘텐츠

  [1/2] 품질 검증 중...
  ✅ 검증 완료
     품질 점수: 8.5/10
     상태: approved

  [2/2] State 업데이트 중...
  ✅ Step 4 완료
  ✅ 전체 파이프라인 완료!

  [파일 저장]
  📁 저장: runs/run_c6a2a43f/step_04.json

  [최

KeyboardInterrupt: 